# kaggle-titanic

* タイトル
    * [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/overview)
* 内容
    * 「この挑戦では、乗客データ(名前、年齢、性別、社会経済的階級など)を使用して、「どのような種類の人々が生き残る可能性が高いか」という質問に答える予測モ*デルを構築するよう求めます。」
* ターゲット
    * 生存するか否か
* 評価指標
    * 変数0(死亡)か1(生存)かの分類

# ライブラリ

In [657]:
import pandas as pd
import numpy as np

#可視化用

#学習予測用
from sklearn.model_selection import train_test_split
import lightgbm as lgb


# データの確認

In [658]:
#トレ―ニングデータの確認
df_train = pd.read_csv('train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [659]:
#テストデータの確認
df_test = pd.read_csv('test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [660]:
#データ数の確認
print('トレーニングデータ数')
print(df_train.shape)
print('テストデータ数')
print(df_test.shape)

トレーニングデータ数
(891, 12)
テストデータ数
(418, 11)


In [661]:
#トレーニングデータ　欠損値、Dtypeの確認
print('トレーニングデータ')
print(df_train.info())

トレーニングデータ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [662]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

*  0   PassengerId  891 non-null    int64  
    * 乗客ID
    * 欠損値なし
*  1   Survived     891 non-null    int64  
    * 生存か否か
    * 死亡: 0, 生存: 1
    * 欠損値なし
*  2   Pclass       891 non-null    int64  
    * チケットランク
    * 1等: 1, 2等: 2, 3等: 3
    * 欠損値なし
*  3   Name         891 non-null    object 
    * 名前
    * 欠損値なし
* 4   Sex          891 non-null    object 
    * 性別
    * 男性: male, 女性: Fmale
    * 欠損値なし
    * **エンコードの必要あり**
* 5   Age          714 non-null    float64
    * 年齢
    * **欠損値あり**
* 6   SibSp        891 non-null    int64
    * 同乗している兄弟、配偶者の数
    * 欠損値なし
* 7   Parch        891 non-null    int64 
    * 同乗している親、子供の数
    * 欠損値なし
* 8   Ticket       891 non-null    object 
    * チケット番号
    * 欠損値なし
* 9   Fare         891 non-null    float64
    * 料金
    * 欠損値なし
* 10  Cabin        204 non-null    object 
    * 客室番号
    * 欠損値ありだが予測に使用しないため無視
* 11  Embarked     889 non-null    object 
    * 出港地
    * C: Cherbourg, Q: Queenstown, S: Southampt

In [663]:
#テストデータ　欠損値、Dtypeの確認
print('テストデータ')
print(df_test.info())

テストデータ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB
None


In [664]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

*  0   PassengerId  418 non-null    int64  
    * 欠損値なし
*  1   Survived     418 non-null    int64  
    * 欠損値なし
*  2   Pclass       418 non-null    int64  
    * 欠損値なし
*  3   Name         418 non-null    object 
    * 欠損値なし
* 4   Sex          418 non-null    object 
    * 欠損値なし
    * **エンコードの必要あり**
* 5   Age          332 non-null    float64
    * **欠損値あり**
    * **前処理必要あり**
* 6   SibSp        418 non-null    int64
    * 欠損値なし
* 7   Parch        418 non-null    int64 
    * 欠損値なし
* 8   Ticket       418 non-null    object 
    * チケット番号
    * 欠損値なし
* 9   Fare         417 non-null    float64
    * 料金
    * **欠損値あり**
* 10  Cabin        91 non-null    object 
    * 客室番号
    * 欠損値ありだが予測に使用しないため無視
* 11  Embarked     418 non-null    object 
    * 出港地
    * **エンコードの必要あり**

In [665]:
#基本統計量の確認
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


---

# 前処理

## Sex

object型のためSexをmale→0,female→1にエンコード

In [666]:
def transform_sex(df):
    pd.set_option('future.no_silent_downcasting', True) #将来のダウンキャスト動作にオプトイン
    df.replace({'male': 0, 'female': 1}, inplace=True)
    df['Sex'] = df['Sex'].astype(int)
    return df

df_train = transform_sex(df_train)
df_test = transform_sex(df_test)

In [667]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [668]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S


## Age

欠損値ありのため、補完する。

In [669]:
#中央値での補完
def transform_age_median(df):
    df['Age'] = df['Age'].fillna(df['Age'].median())
    return df

df_train = transform_age_median(df_train)
df_test = transform_age_median(df_test)

In [670]:
'''#平均値での補完
def transform_age_mean(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

df_train = transform_age_mean(df_train)
df_test = transform_age_mean(df_test)'''

"#平均値での補完\ndef transform_age_mean(df):\n    df['Age'] = df['Age'].fillna(df['Age'].mean())\n    return df\n\ndf_train = transform_age_mean(df_train)\ndf_test = transform_age_mean(df_test)"

In [671]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [672]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Fare

欠損値ありのため補完する。

In [673]:
#中央値での補完
def transform_age_median(df):
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    return df

df_train = transform_age_median(df_train)
df_test = transform_age_median(df_test)

In [674]:
'''#平均値での補完
def transform_age_mean(df):
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df

df_train = transform_age_mean(df_train)
df_test = transform_age_mean(df_test)'''

"#平均値での補完\ndef transform_age_mean(df):\n    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())\n    return df\n\ndf_train = transform_age_mean(df_train)\ndf_test = transform_age_mean(df_test)"

In [675]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [676]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

## Embarked

トレインデータに欠損値ありのため０として保管する。

In [677]:
df_train.query('Embarked != Embarked')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",1,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",1,62.0,0,0,113572,80.0,B28,NaN


In [678]:
df_train['Embarked'].replace(np.nan, 0, inplace=True)


C:\Users\whand\AppData\Local\Temp\ipykernel_13072\2784580491.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Embarked'].replace(np.nan, 0, inplace=True)


In [679]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

C: Cherbourg, Q: Queenstown, S: Southampt<br>
をそれぞれ1,2,3へ置換する。


In [680]:
def transform_embarked(df):
    pd.set_option('future.no_silent_downcasting', True)
    df.replace({'C': 1, 'Q': 2, 'S': 3}, inplace=True)
    df['Embarked'] = df['Embarked'].astype(int)
    return df

df_train = transform_embarked(df_train)
df_test = transform_embarked(df_test)

In [681]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,3
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,3


In [682]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,3
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,2
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,3
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,3


In [683]:
#不要列の削除
def delete_df_column(df):
    return df.drop(['Name', 'Cabin', 'Ticket'], axis=1)

df_train = delete_df_column(df_train)
df_test = delete_df_column(df_test)

In [684]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,3
1,2,1,1,1,38.0,1,0,71.2833,1
2,3,1,3,1,26.0,0,0,7.9250,3
3,4,1,1,1,35.0,1,0,53.1000,3
4,5,0,3,0,35.0,0,0,8.0500,3


In [685]:
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,2
1,893,3,1,47.0,1,0,7.0000,3
2,894,2,0,62.0,0,0,9.6875,2
3,895,3,0,27.0,0,0,8.6625,3
4,896,3,1,22.0,1,1,12.2875,3


In [686]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int32  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    int32  
dtypes: float64(2), int32(2), int64(5)
memory usage: 55.8 KB


---

# 可視化

---

# 予測モデルの学習

LGBMでの予測

In [687]:
#説明変数 X と目的変数 y に分ける
X_train = df_train.drop(['PassengerId', 'Survived'], axis=1)
y_train = df_train['Survived']

X_test = df_test.drop(['PassengerId'], axis=1)

In [688]:
# X_trainをX_train（学習用）とX_valid（検証用）に分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

# カテゴリカル変数の明示
categorical_features = ['Sex', 'Pclass', 'Embarked']

# LightGBMでの学習予測
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {
    'objective': 'binary',
    'verbose': 10, 
    'early_stopping_rounds': 10
}

model = lgb.train(params,
                  lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  num_boost_round=1000)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.706661
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.406329
[LightGBM] [Debug] init for col-wise cost 0.000445 seconds, init for row-wise cost 0.001764 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 22 and depth = 8
Trainin

一回目　[22]	training's binary_logloss: 0.335206	valid_1's binary_logloss: 0.437223　submissionscore:0.76794<br>
二回目　[20]	training's binary_logloss: 0.341354	valid_1's binary_logloss: 0.437472　submissionscore:0.76315 低下(Ageの欠損値を中央値から平均値へ変更)<br>

In [689]:
y_pred[:10]

array([0.06799741, 0.53092879, 0.12402525, 0.10202349, 0.39770746,
       0.49337154, 0.70464621, 0.13773163, 0.68496324, 0.0487823 ])

In [690]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0])

---

# 提出用データ

In [691]:
df_pred = pd.DataFrame(y_pred, columns=['Survived'])

In [692]:
df_pred.head()

,Survived
0,0
1,1
2,0
3,0
4,0


In [693]:
df_test = pd.concat([df_test, df_pred], axis=1)

In [694]:
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,0,34.5,0,0,7.8292,2,0
1,893,3,1,47.0,1,0,7.0000,3,1
2,894,2,0,62.0,0,0,9.6875,2,0
3,895,3,0,27.0,0,0,8.6625,3,0
4,896,3,1,22.0,1,1,12.2875,3,0


In [695]:
submission = df_test[['PassengerId', 'Survived']]

In [696]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [697]:
#submission.to_csv('submission.csv', index=False)